In [1]:
print("TODO: Please paste the gpt-4_pair.jsonl and gpt-4_single.jsonl files into the data folder.")

TODO: Please paste the gpt-4_pair.jsonl and gpt-4_single.jsonl files into the data folder.


In [2]:
!pip install -U plotly kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.1 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [3]:
use_pair_comparison = True

In [4]:
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


CATEGORIES = ["Writing", "Roleplay", "Reasoning", "Math", "Coding", "Extraction", "STEM", "Humanities"]


def get_model_df():
    cnt = 0
    q2result = []
    fin = open("gpt-4_single.jsonl", "r")
    for line in fin:
        obj = json.loads(line)
        obj["category"] = CATEGORIES[(obj["question_id"]-81)//10]
        q2result.append(obj)
    df = pd.DataFrame(q2result)
    return df

def toggle(res_str):
    if res_str == "win":
        return "loss"
    elif res_str == "loss":
        return "win"
    return "tie"

def get_model_df_pair():
    fin = open("gpt-4_pair.jsonl", "r")
    cnt = 0
    q2result = []
    for line in fin:
        obj = json.loads(line)

        result = {}
        result["qid"] = str(obj["question_id"])
        result["turn"] = str(obj["turn"])
        if obj["g1_winner"] == "model_1" and obj["g2_winner"] == "model_1":
            result["result"] = "win"
        elif obj["g1_winner"] == "model_2" and obj["g2_winner"] == "model_2":
            result["result"] = "loss"
        else:
            result["result"] = "tie"
        result["category"] = CATEGORIES[(obj["question_id"]-81)//10]
        result["model"] = obj["model_1"]
        result["baseline"] = obj["model_2"]
        q2result.append(result)

    df = pd.DataFrame(q2result)

    return df

df = get_model_df()

if use_pair_comparison:
  df_pair = get_model_df_pair()

In [5]:
# Look for duplicates and keep only the one with the newest timestamp

df.judge = df.judge.apply(lambda x: tuple(x) if type(x)!= str else tuple([x]))

def drop_duplicates(df_):
  duplicates = df_[df_.duplicated(subset=['model','question_id', 'judge'], keep=False)]

  for index, row in duplicates.iterrows():
    dff = df_[(df_.model == row.model) & (df_.question_id == row.question_id) & (df_.judge == row.judge)]
    index_old_values = dff[dff.tstamp < dff.tstamp.max()].index
    df_.drop(index_old_values, inplace=True)

  return df_

print(len(df))
df = drop_duplicates(df)
print(len(df))

8182
8000


In [6]:
all_models = df["model"].unique()
print(all_models)
scores_all = []
for model in all_models:
    for cat in CATEGORIES:
        # filter category/model, and score format error (<1% case)
        res = df[(df["category"]==cat) & (df["model"]==model) & (df["score"] >= 0)]
        score = res["score"].mean()

        if use_pair_comparison and len(df_pair[(df_pair["category"]==cat) & (df_pair["model"]==model)]["baseline"].unique()) > 0:
          # pairwise result
          for baseline in df_pair[(df_pair["category"]==cat) & (df_pair["model"]==model)]["baseline"].unique():
            res_pair = df_pair[(df_pair["category"]==cat) & (df_pair["model"]==model) & (df_pair["baseline"] == baseline)]["result"].value_counts()
            wincnt = res_pair["win"] if "win" in res_pair.index else 0
            tiecnt = res_pair["tie"] if "tie" in res_pair.index else 0
            winrate = wincnt/res_pair.sum()
            winrate_adjusted = (wincnt + tiecnt)/res_pair.sum()
            winrate_adjusted_sage = (wincnt + 0.5*tiecnt)/res_pair.sum()

            scores_all.append({"model": model, "category": cat, "score": score, "winrate": winrate, "wtrate": winrate_adjusted, "wtrate_sage": winrate_adjusted_sage, "baseline": baseline})

        else:
          scores_all.append({"model": model, "category": cat, "score": score})

['alpaca-13b' 'baize-v2-13b' 'chatglm-6b' 'claude-instant-v1' 'claude-v1'
 'dolly-v2-12b' 'falcon-40b-instruct' 'fastchat-t5-3b' 'gpt-3.5-turbo'
 'gpt-4' 'gpt4all-13b-snoozy' 'guanaco-33b' 'guanaco-65b'
 'h2ogpt-oasst-open-llama-13b' 'koala-13b' 'llama-13b' 'mpt-30b-chat'
 'mpt-30b-instruct' 'mpt-7b-chat' 'nous-hermes-13b'
 'oasst-sft-4-pythia-12b' 'oasst-sft-7-llama-30b' 'palm-2-chat-bison-001'
 'rwkv-4-raven-14b' 'stablelm-tuned-alpha-7b' 'tulu-30b' 'vicuna-13b-v1.3'
 'vicuna-33b-v1.3' 'vicuna-7b-v1.3' 'wizardlm-13b' 'wizardlm-30b'
 'Llama-2-7b-chat' 'Llama-2-13b-chat' 'Llama-2-70b-chat' 'vicuna-7b-v1.5'
 'vicuna-7b-v1.5-awq-activate_4bit-deactivate_nested-float16-nf4'
 'tugg-merged_final_checkpoints_lora_0'
 'tugg-merged_final_checkpoints_adalora_0'
 'tugg-merged_final_checkpoints_adalora_1'
 'tugg-merged_final_checkpoints_lokr_0'
 'tugg-merged_final_checkpoints_lora_1'
 'vicuna-7b-v1.5-no-deactivate_4bit-deactivate_nested-float16-nf4'
 'vicuna-7b-v1.5-gptq-deactivate_4bit-deactivat

In [7]:
rename_map = {"gpt-4": "GPT-4",
              "Llama-2-7b-no-deactivate_4bit-deactivate_nested-float16-nf4": "LLamaV2-7B Baseline",
              'tugg-merged_final_checkpoints_lora_0': "LlamaV2-7B Lora Finetuned",
              'tugg-merged_final_checkpoints_adalora_0': "LlamaV2-7B AdaLora Finetuned",
              'tugg-merged_final_checkpoints_adalora_1': "LlamaV2-7B Q-AdaLora Finetuned",
              'tugg-merged_final_checkpoints_lokr_0': "LlamaV2-7B Lokr Finetuned",
              'tugg-merged_final_checkpoints_lora_1': "LlamaV2-7B Q-Lora Finetuned",
              'vicuna-7b-v1.5-no-deactivate_4bit-deactivate_nested-float16-nf4': "Vicuna-7B",
              "vicuna-7b-v1.5-awq-activate_4bit-deactivate_nested-float16-nf4": "Vicuna-7B AWQ",
              'vicuna-7b-v1.5-gptq-deactivate_4bit-deactivate_nested-float16-nf4': "Vicuna-7B GPTQ",
              'vicuna-7b-v1.5-bnb-activate_4bit-deactivate_nested-float16-fp4': "Vicuna-7B BnB",
              "Llama-2-7b-no-deactivate_4bit-deactivate_nested-float16-nf4": "LLamaV2-7B Baseline",
              'Llama-2-7b-chat-no-deactivate_4bit-deactivate_nested-float16-nf4': "LLamaV2-7B-Chat Baseline",
              'Llama-2-7b-chat-bnb-activate_4bit-deactivate_nested-float16-fp4': "LLamaV2-7B-Chat BnB",
              'Llama-2-7b-chat-bnb-activate_4bit-activate_nested-float16-fp4': "LLamaV2-7B-Chat nested BnB",
              'Llama-2-7b-chat-gptq-activate_4bit-deactivate_nested-float16-nf4': "LLamaV2-7B-Chat GPTQ",
              'Llama-2-7b-chat-awq-activate_4bit-deactivate_nested-float16-nf4': "LLamaV2-7B-Chat AWQ"
              }

In [8]:
def create_polar_plot(target_models_single):
  scores_target_single = [scores_all[i] for i in range(len(scores_all)) if scores_all[i]["model"] in target_models_single]

  # sort by target_models
  scores_target_single = sorted(scores_target_single, key=lambda x: target_models_single.index(x["model"]), reverse=True)

  df_score_single = pd.DataFrame(scores_target_single)
  df_score_single = df_score_single[df_score_single["model"].isin(target_models_single)]

  for k, v in rename_map.items():
      df_score_single.replace(k, v, inplace=True)

  fig = px.line_polar(df_score_single, r = 'score', theta = 'category', line_close = True, category_orders = {"category": CATEGORIES},
                      color = 'model', markers=True, color_discrete_sequence=px.colors.qualitative.Pastel)

  fig.show()

  return fig, df_score_single


In [9]:
def store_figure(fig, figname):
    fig.update_layout(
      font=dict(
          size=18,
      ),
    )
    fig.write_image(figname, width=1200, height=600, scale=2)

In [10]:
target_models_single = ["gpt-4",
                        "Llama-2-7b-no-deactivate_4bit-deactivate_nested-float16-nf4",
                        'tugg-merged_final_checkpoints_lora_0',
                        'tugg-merged_final_checkpoints_adalora_0',
                        'tugg-merged_final_checkpoints_adalora_1',
                        'tugg-merged_final_checkpoints_lokr_0',
                        'tugg-merged_final_checkpoints_lora_1']

fig, df_single_1_scores = create_polar_plot(target_models_single)
store_figure(fig, "single_1.png")
df_single_1_scores.to_csv("single_scores_1.csv", index=False)


In [11]:
target_models_single = ["gpt-4",
                        'Llama-2-7b-no-deactivate_4bit-deactivate_nested-float16-nf4',
                        'Llama-2-7b-chat-no-deactivate_4bit-deactivate_nested-float16-nf4',
                        'Llama-2-7b-chat-bnb-activate_4bit-deactivate_nested-float16-fp4',
                        'Llama-2-7b-chat-bnb-activate_4bit-activate_nested-float16-fp4',
                        'Llama-2-7b-chat-gptq-activate_4bit-deactivate_nested-float16-nf4',
                        'Llama-2-7b-chat-awq-activate_4bit-deactivate_nested-float16-nf4'
                        ]

fig, df_single_2_scores = create_polar_plot(target_models_single)
store_figure(fig, "single_2.png")
df_single_2_scores.to_csv("single_scores_2.csv", index=False)


In [12]:
def create_pair_polar_plot(target_models_pair, baseline):
  scores_target_pair = [scores_all[i] for i in range(len(scores_all)) if scores_all[i]["model"] in target_models_pair and scores_all[i]["baseline"] == baseline]

  baselines = [b['baseline'] for b in scores_target_pair]
  assert len(set(baselines)) == 1, "Pair Plot only possible for models using the same baseline"

  # sort by target_models
  scores_target_pair = sorted(scores_target_pair, key=lambda x: target_models_pair.index(x["model"]), reverse=True)

  df_score_pair = pd.DataFrame(scores_target_pair)
  df_score_pair = df_score_pair[df_score_pair["model"].isin(target_models_pair)]

  for k, v in rename_map.items():
      df_score_pair.replace(k, v, inplace=True)


  fig = px.line_polar(df_score_pair, r = 'wtrate_sage', theta = 'category', line_close = True, category_orders = {"category": CATEGORIES},
                      color = 'model', title=f"Pair Comparison vs. {rename_map[baselines[0]]}", markers=True, color_discrete_sequence=px.colors.qualitative.Pastel, range_r=[0.0, 1.0])
  fig.show()
  return fig, df_score_pair

In [16]:
target_models_pair = ['tugg-merged_final_checkpoints_adalora_0', 'tugg-merged_final_checkpoints_adalora_1',
                 'tugg-merged_final_checkpoints_lokr_0', 'tugg-merged_final_checkpoints_lora_1']

fig, df_pair_scores = create_pair_polar_plot(target_models_pair, baseline='tugg-merged_final_checkpoints_lora_0')
store_figure(fig, "pair_1.png")

In [17]:
df_pair_scores.to_csv("pair_scores_1.csv", index=False)

In [15]:
target_models_pair = ['tugg-merged_final_checkpoints_lora_0', 'tugg-merged_final_checkpoints_adalora_0', 'tugg-merged_final_checkpoints_adalora_1',
                 'tugg-merged_final_checkpoints_lokr_0', 'tugg-merged_final_checkpoints_lora_1']

fig, df_pair_scores = create_pair_polar_plot(target_models_pair, baseline='Llama-2-7b-no-deactivate_4bit-deactivate_nested-float16-nf4')
store_figure(fig, "pair_2.png")
df_pair_scores.to_csv("pair_scores_2.csv", index=False)